In [2]:
import os
# os.environ['ALFRED_ROOT'] = '/root/data/home/hoyeung/alfred/'
os.environ['ALFRED_ROOT'] = '/root/home/legg/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
import numpy as np
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from collections import Counter
# from models.utils.helper_utils import optimizer_to

In [4]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
# args.data = '/root/data_alfred/json_data_augmentation_20200820'
# args.data = '/media/legg/data/alfred_data/json_data_augmentation_20200820'
args.pp_folder = 'pp'
args.pframe = 300
args.fast_epoch = True

## Preprocess original train splits oct21

In [5]:
# SKIP preprocessing. Already preprocessed

## Preprocess augmented splits

In [3]:
vocab = torch.load(os.path.join('/media/legg/data/alfred_data/json_feat_2.1.0_backup_20200826_agent_training', "%s.vocab" % args.pp_folder))

In [4]:
vocab

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}

In [26]:
args.splits = '/root/data_alfred/splits/sample_failed_20200820_filtered.json'
# args.splits = '/root/data_alfred/splits/sample_failed_toy.json'

with open(args.splits, 'r') as f:
    splits = json.load(f)

print([(k,len(splits[k])) for k in splits.keys()])

[('augmentation', 7570)]


In [27]:
splits['augmentation'][0]

{'task': 'look_at_obj_in_light-BaseballBat-None-DeskLamp-301/trial_T20200814_164125_595727',
 'repeat_idx': 0,
 'full_traj_success': False,
 'collected_subgoals': 2}

In [28]:
args.fast_epoch = False
dataset = Dataset(args, vocab)
dataset.preprocess_splits_augmentation(splits, baseline=True)

100% (7570 of 7570) |####################| Elapsed Time: 0:01:38 Time:  0:01:38


In [29]:
args.fast_epoch = False
dataset = Dataset(args, vocab)
dataset.preprocess_splits_augmentation(splits, baseline=False)

100% (7570 of 7570) |####################| Elapsed Time: 0:01:29 Time:  0:01:29


# Make SPLITS

## Read in existing splits

In [6]:
original_alfred_splits_p = '/root/home/legg/data_alfred/splits/oct21.json'

with open(original_alfred_splits_p, 'r') as f:
    original_alfred_splits = json.load(f)
    
print([(k,len(original_alfred_splits[k])) for k in original_alfred_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821)]


In [8]:
augmented_splits_p = '/root/home/legg/data_alfred/splits/sample_failed_20200820_filtered.json'

with open(augmented_splits_p, 'r') as f:
    augmented_splits = json.load(f)

print([(k,len(augmented_splits[k])) for k in augmented_splits.keys()])

[('augmentation', 7570)]


## Identify Unseen envs to be filtered out from failed trajectories

In [21]:
# Provided by Mohit
envs_train = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 220, 221, 222, 223, 224, 225, 227, 228, 229, 230, 301, 302, 303, 304, 305, 306, 307, 309, 310, 311, 312, 313, 314, 316, 317, 318, 319, 320, 321, 322, 323, 324, 326, 327, 328, 329, 330, 401, 402, 403, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 426, 427, 428, 429, 430]
envs_valid_seen = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 201, 202, 203, 204, 205, 206, 207, 212, 213, 214, 216, 218, 222, 223, 224, 225, 227, 229, 230, 301, 302, 303, 304, 305, 309, 310, 311, 313, 314, 316, 318, 320, 323, 324, 326, 327, 328, 329, 330, 401, 402, 403, 405, 406, 407, 408, 409, 410, 412, 413, 414, 415, 417, 418, 419, 422, 423, 426, 427, 428, 429]
envs_valid_unseen = [10, 219, 308, 424]
envs_test_seen = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 220, 221, 222, 223, 224, 225, 227, 228, 230, 301, 302, 303, 304, 305, 306, 307, 309, 310, 311, 312, 313, 314, 316, 317, 318, 319, 320, 321, 322, 323, 324, 326, 327, 328, 329, 330, 401, 402, 403, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 426, 427, 428, 429, 430]
envs_test_unseen = [215, 226, 29, 315, 325, 404, 425, 9]

envs_train = set(envs_train)
envs_valid_seen = set(envs_valid_seen)
envs_valid_unseen = set(envs_valid_unseen)
envs_test_seen = set(envs_test_seen)
envs_test_unseen = set(envs_test_unseen)

In [29]:
envs_augmentation = []

for task in augmented_splits['augmentation']:
    env_id = task['task'].split('/')[-2].split('-')[-1]
    envs_augmentation.append(int(env_id))

envs_augmentation = set(envs_augmentation)
len(envs_augmentation)

120

In [32]:
envs_augmentation & envs_valid_unseen

{10, 219, 308, 424}

In [33]:
envs_augmentation & envs_test_unseen

{9, 29, 215, 226, 315, 325, 404, 425}

In [53]:
len(envs_valid_unseen|envs_test_unseen)

12

## 1. Annotate all failures with Explainer and with Baseline.  Retrain agent and evaluate

In [50]:
def filter_unseen(split, unseen_env_ids):
    out = []
    for task in split:
        env_id = int(task['task'].split('/')[-2].split('-')[-1])
        if not env_id in unseen_env_ids:
            out.append(task)
    return out

In [52]:
experiment_1_splits = {}

for k in original_alfred_splits.keys():
    experiment_1_splits[k] = original_alfred_splits[k] 

experiment_1_splits['augmentation'] = filter_unseen(
    split=augmented_splits['augmentation'], unseen_env_ids=envs_valid_unseen|envs_test_unseen)

print([(k,len(experiment_1_splits[k])) for k in experiment_1_splits.keys()])

# before correction:
# [('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821), 
# ('augmentation', 7570)]

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821), ('augmentation', 6949)]


In [55]:
experiment_1_split_path = '/root/home/legg/data_alfred/splits/data_augmentation_experiment1_20200826_redo20210608.json'
with open(experiment_1_split_path, 'w') as f:
    json.dump(experiment_1_splits, f)
    
with open(experiment_1_split_path, 'r') as f:
    dum = json.load(f)

print([(k,len(dum[k])) for k in dum.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821), ('augmentation', 6949)]


# Ignore below

## 2. Retrain agent on only half the original training data and evaluate 

In [29]:
# distribution -- number of annotations per task
counter = dict()

for task in original_alfred_splits['train']:
    if task['task'] not in counter:
        counter[task['task']] = 0
    counter[task['task']] += 1
    
print(Counter(counter.values()))
print(sum([k*v for k,v in Counter(counter.values()).items()]))
print(len(counter.keys()))

Counter({3: 5990, 6: 282, 5: 156, 4: 145, 1: 1})
21023
6574


In [30]:
# randomly select half of the tasks
np.random.seed(42)
keep_tasks = np.random.choice(list(counter.keys()), size=len(counter)//2, replace=False)
print(keep_tasks[:10])
keep_tasks = set(keep_tasks)
print(len(keep_tasks))

['pick_and_place_simple-WineBottle-None-Shelf-7/trial_T20190907_200104_945677'
 'pick_cool_then_place_in_recep-Bowl-None-Cabinet-24/trial_T20190909_112909_923737'
 'pick_heat_then_place_in_recep-AppleSliced-None-DiningTable-17/trial_T20190908_095621_618627'
 'pick_and_place_simple-TennisRacket-None-SideTable-310/trial_T20190907_040920_796378'
 'pick_two_obj_and_place-CreditCard-None-Desk-313/trial_T20190909_104526_715846'
 'pick_and_place_simple-Candle-None-Cart-430/trial_T20190909_055319_119903'
 'pick_heat_then_place_in_recep-PotatoSliced-None-Fridge-22/trial_T20190908_185723_946017'
 'pick_and_place_with_movable_recep-RemoteControl-Box-Sofa-220/trial_T20190907_023003_992808'
 'pick_heat_then_place_in_recep-Egg-None-SinkBasin-25/trial_T20190907_024723_698757'
 'pick_cool_then_place_in_recep-Plate-None-Cabinet-25/trial_T20190908_102731_668233']
3287


In [33]:
# create new train split
half_train_split = []
ct = 0
for task in original_alfred_splits['train']:
    if task['task'] in keep_tasks:
        ct += 1
        half_train_split.append(task)
        
print(ct)

10524


In [147]:
# assemble splits
experiment_2_splits = {}

for k in original_alfred_splits.keys():
    if k != 'train':
        experiment_2_splits[k] = original_alfred_splits[k] 

experiment_2_splits['train'] = half_train_split

print([(k,len(experiment_2_splits[k])) for k in experiment_2_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('valid_seen', 820), ('valid_unseen', 821), ('train', 10524)]


In [151]:
# save out
experiment_2_split_path = '/media/legg/data/alfred_data/splits/data_augmentation_experiment2_20200831.json'

with open(experiment_2_split_path, 'w') as f:
    json.dump(experiment_2_splits, f)
    
with open(experiment_2_split_path, 'r') as f:
    dum = json.load(f)

print([(k,len(dum[k])) for k in dum.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('valid_seen', 820), ('valid_unseen', 821), ('train', 10524)]


## 3. Label the other half of the training data with the Explainer, retrain agent, eval

### 3.1 Get list of tasks that were labeled by the explainer

In [20]:
dout = '/media/legg/data/alfred_data/exp_all/model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2_sample_sentences/'

split = 'train'
temperature = '0.75'
pred_f = os.path.join(dout, '{}_sampled.temperature_{}.preds.json'.format(split, temperature))
with open(pred_f, 'r') as f:
    explainer_pred = json.load(f)

In [21]:
pred_available_list_exp = []

for k in explainer_pred.keys():
    tk = '/'.join(explainer_pred[k]['root'].split('/')[-2:])
    pred_available_list_exp.append(tk)

In [22]:
dout = '/media/legg/data/alfred_data/exp_all/model_seq2seq_per_subgoal,name_v2_epoch_35_baseline_sample_sentences/'

split = 'train'
temperature = '0.75'
pred_f = os.path.join(dout, '{}_sampled.temperature_{}.preds.json'.format(split, temperature))
with open(pred_f, 'r') as f:
    baseline_pred = json.load(f)

In [23]:
pred_available_list_base = []

for k in baseline_pred.keys():
    tk = '/'.join(baseline_pred[k]['root'].split('/')[-2:])
    pred_available_list_base.append(tk)

In [26]:
len(set(pred_available_list_exp))

6505

In [25]:
len(set(pred_available_list_exp))

6505

### 3.2 Filter out the half used in experiment 2; But labeled by Explainer

In [31]:
# create new train split
other_half_train_split = []
ct = 0
for task in original_alfred_splits['train']:
    if (task['task'] not in keep_tasks) and (task['task'] in set(pred_available_list_exp)):
        ct += 1
        task['use_autogeneration'] = True
        other_half_train_split.append(task)

print(ct)

10394


### 3.3 Merge the half used in experiment 2 and the other half we tagged as autogenerated

In [35]:
print(len(half_train_split))
print(len(other_half_train_split))

10524
10394


In [41]:
# assert no overlap
assert not set([t['task'] for t in half_train_split]) & set([t['task'] for t in other_half_train_split])

set()

In [36]:
# assemble splits
experiment_3_splits = {}

for k in original_alfred_splits.keys():
    if k != 'train':
        experiment_3_splits[k] = original_alfred_splits[k] 

experiment_3_splits['train'] = half_train_split + other_half_train_split

print([(k,len(experiment_3_splits[k])) for k in experiment_3_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('valid_seen', 820), ('valid_unseen', 821), ('train', 20918)]


In [38]:
# save out
experiment_3_split_path = '/media/legg/data/alfred_data/splits/data_augmentation_experiment3_20200903.json'

# original_alfred_splits has been modified inplace
with open(experiment_3_split_path, 'w') as f:
    json.dump(experiment_3_splits, f)
    
with open(experiment_3_split_path, 'r') as f:
    dum = json.load(f)

print([(k,len(dum[k])) for k in dum.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('valid_seen', 820), ('valid_unseen', 821), ('train', 20918)]


# Ignore below

In [36]:
p = '/media/legg/data/alfred_data/json_feat_2.1.0_backup_20200826_agent_training'
# data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'aug_explainer_0_T20200810_1829.json')
data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'ann_0.json')


with open(data_p, 'r') as f:
    data1 = json.load(f)

In [37]:
p = '/media/legg/data/alfred_data/json_feat_2.1.0_backup_20200826_agent_training'
data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'aug_explainer_0_T20200810_1829.json')
# data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'ann_0.json')


with open(data_p, 'r') as f:
    data2 = json.load(f)